# Library Features and Implementation: for catalog, employee, and patron collections

In [1]:
import pymongo as pym
import redis
import json
from bson.son import SON

In [2]:
client = pym.MongoClient("mongodb+srv://<username>:<password>@librarydb.tec4i8b.mongodb.net/?retryWrites=true&w=majority")
r = redis.Redis(
  host='<YOUR HOST HERE>',
  port=10522,
  password='YOUR PASSWORD HERE')
db = client['libraryDB']
catalog = db.books
patrons = db.patrons
employees = db.employees
texts = db.texts

## History

In [3]:
def save_to_history(collection, doc_id, query):
    doc_history = f'{collection}_ID{doc_id}_History'
    count = r.zcard(doc_history)
    
    doc_dict = {}
    doc_dict[query] = count + 1
    
    r.zadd(doc_history, doc_dict)

In [4]:
def undo(collection, doc_id):
    doc_history = f'{collection}_ID{doc_id}_History'
    doc = collection.find_one(doc_id)
    
    try:
        count = r.zcard(doc_history) + 1
        query = {'id': doc_id}
        value = r.zrange(doc_history, count, count + 1)
        value = str(value)
        
        r.zremrangebyscore(doc_history, count, count + 1)
    except:
        print(f'{doc} does not have an edit history')

In [5]:
def get_history(collection, doc_id):
    doc_history = f'{collection}_ID{doc_id}_History'
    return r.zrange(doc_history, 0, -1)

### Demo

In [6]:
def update_book_field(book_id, field, value):
        query = {'_id': book_id}
        set_value = {'$set': {field : value}}
        
        save_to_history(catalog, book_id, set_value)
        catalog.update_one(query, set_value)

In [7]:
def create_popUp_bookClub(creator, club_name):
    club_name = f'The {club_name} Club'
    if r.exists(club_name):
        print(f'{club_name} already exists.')
    else:
        nearby = find_nearby_patrons(creator['_id'])[0]
        count = r.zcard(club_name)

        club_dict = {}
        club_dict[f'{count + 1}. Patron {creator["firstName"]} {creator["surname"]} has created {club_name}!'] = count + 1

        r.zadd(club_name, club_dict)
        for recipient_id in nearby:
            recipient = get_patron(recipient_id)
            send_message(creator, recipient, f'Just created {club_name}! Setting up at {creator["loc"]}')

## Universal Search

In [8]:
def search_by_field(collection, key, value, find_one):
    query = {key: value}
    if find_one == True:
        doc = collection.find_one(query)
    else:
        doc = list(collection.find(query))
    return doc

### Demo

In [9]:
search_by_field(catalog, 'author', 'Mark Twain', True)

{'_id': 2965,
 'title': 'The Wit and Wisdom of Mark Twain',
 'author': 'Mark Twain',
 'average_rating': 4.2,
 'isbn': '486406644',
 'language_code': 'eng',
 'num_pages': 64,
 'ratings_count': 970,
 'text_reviews_count': 53,
 'publication_date': '12/23/1998',
 'publisher': 'Dover Publications',
 'available': 1}

In [10]:
search_by_field(patrons, 'surname', 'Roberts', False)

[{'_id': 395,
  'surname': 'Roberts',
  'firstName': 'Clayton',
  'address': '4425 Gentry Vista, Roscoe, IL, 61080',
  'phone': '(822)867-4049',
  'email': 'clroberts@gmail.com',
  'username': 'clroberts',
  'password': 'BQCIytZ31JQh',
  'loc': [-89.02997287891019, 42.41248832044677],
  'pubPriv': 'private',
  'checkedOutBooks': [],
  'favorites': []},
 {'_id': 638,
  'surname': 'Roberts',
  'firstName': 'Brandi',
  'address': '6163 Kristen Course, Roscoe, IL, 61080',
  'phone': '(915)733-0943',
  'email': 'brroberts@yahoo.com',
  'username': 'brroberts',
  'password': 'BSOMwdMOMkAsQZBBkpXCuiP3',
  'loc': [-89.0436540622616, 42.42096353554921],
  'pubPriv': 'public',
  'checkedOutBooks': [],
  'favorites': []},
 {'_id': 924,
  'surname': 'Roberts',
  'firstName': 'Lisa',
  'address': '6359 Shah Green Suite 754, Roscoe, IL, 61080',
  'phone': '(955)093-3709',
  'email': 'liroberts@yahoo.com',
  'username': 'liroberts',
  'password': 'srS9YJNPv6z6dmC6',
  'loc': [-89.01545969979794, 42.3

In [11]:
search_by_field(employees, 'weeklyHours', 40, False)

[{'_id': 1,
  'surname': 'Raymond',
  'firstName': 'Kim',
  'schedule': 'C',
  'weeklyHours': 40,
  'wageType': 'Salary',
  'pay': 48037.68,
  'phone': '926-138-8148',
  'email': 'kiraymond@gmail.com',
  'username': 'kiraymond',
  'password': 'eXGCrZF9lcAGPc7gIgLYGnYF'},
 {'_id': 3,
  'surname': 'Ponce',
  'firstName': 'Matthew',
  'schedule': 'A',
  'weeklyHours': 40,
  'wageType': 'Salary',
  'pay': 69420.44,
  'phone': '402-621-2034',
  'email': 'maponce@gmail.com',
  'username': 'maponce',
  'password': '_IDbFdb72d2KyHI'},
 {'_id': 5,
  'surname': 'Duarte',
  'firstName': 'Lindsay',
  'schedule': 'C',
  'weeklyHours': 40,
  'wageType': 'Salary',
  'pay': 66590.57,
  'phone': '175-182-5955',
  'email': 'liduarte@gmail.com',
  'username': 'liduarte',
  'password': 'twjHddHs9xc_Ya3saWY'},
 {'_id': 6,
  'surname': 'Martin',
  'firstName': 'Margaret',
  'schedule': 'A',
  'weeklyHours': 40,
  'wageType': 'Salary',
  'pay': 67658.12,
  'phone': '871-817-1127',
  'email': 'mamartin@live.c

## OPAC

### Basic SCUM

In [12]:
def get_book(book_id):
    query = {'_id': book_id}
    book = catalog.find_one(query)
    return book

In [13]:
def add_book(book_id, title, author, rating, isbn, language_code,
             num_pages, ratings_count,text_reviews_count, publication_date,
             publisher):
    try:
        this_book = {'_id': book_id,
                    'title': title,
                    'author': author,
                    'rating': rating,
                    'isbn': isbn,
                    'language_code': language_code,
                    'num_pages': num_pages,
                    'ratings_count': ratings_count,
                    'text_reviews_count': text_reviews_count,
                    'publication_date': publication_date,
                    'publisher': publisher,
                    'available': 1}

        catalog.insert_one(this_book)
        print(f'Sucessfully added "{title}" to catalog')
    except:
        print('Book ID already exists in this system')

In [14]:
def add_books(book_list):
    catalog.insert_many(book_list)

In [15]:
def delete_book(book_id):
    try:
        query = {'_id': book_id}
        book = catalog.find_one(query)
        catalog.delete_one(query)
        print(f'Successfully deleted "{book["title"]}"')
    except:
        print(f'Book ID: {book_id} does not exist in our system')

In [16]:
def delete_books(bookID_list):
    for book_id in bookID_list:
        delete_book(book_id)

In [17]:
def update_book_field(book_id, field, value):
        query = {'_id': book_id}
        set_value = {'$set': {field : value}}
        
        save_to_history(catalog, book_id, str(set_value))
        catalog.update_one(query, set_value)

#### Demo

In [18]:
add_book(2,'Sad Times for Silly Men', 'Pam Elliot', 1.1, '754654',
         'eng', 501, 521, 18, '6/1/1991', 'Little Brown Book')

Sucessfully added "Sad Times for Silly Men" to catalog


In [19]:
get_book(2)

{'_id': 2,
 'title': 'Sad Times for Silly Men',
 'author': 'Pam Elliot',
 'rating': 1.1,
 'isbn': '754654',
 'language_code': 'eng',
 'num_pages': 501,
 'ratings_count': 521,
 'text_reviews_count': 18,
 'publication_date': '6/1/1991',
 'publisher': 'Little Brown Book',
 'available': 1}

In [20]:
search_by_field(catalog, 'author', 'Pam Elliot', False)

[{'_id': 2,
  'title': 'Sad Times for Silly Men',
  'author': 'Pam Elliot',
  'rating': 1.1,
  'isbn': '754654',
  'language_code': 'eng',
  'num_pages': 501,
  'ratings_count': 521,
  'text_reviews_count': 18,
  'publication_date': '6/1/1991',
  'publisher': 'Little Brown Book',
  'available': 1}]

In [21]:
update_book_field(2, 'title', 'The Ugliest Boy in Town')

In [22]:
update_book_field(2, 'title', 'Total Recall')

In [23]:
update_book_field(2, 'title', 'Weeble Wobble')

In [24]:
get_history(catalog, 2)

[b"{'$set': {'title': 'The Ugliest Boy in Town'}}",
 b"{'$set': {'title': 'Total Recall'}}",
 b"{'$set': {'title': 'Weeble Wobble'}}"]

In [25]:
undo(catalog, 2)

In [26]:
delete_book(2)

Successfully deleted "Weeble Wobble"


## Troubleshooting functions

In [27]:
def update_book_field(book_id, field, value):
        query = {'_id': book_id}
        set_value = {'$set': {field : value}}
        
        save_to_history(catalog, book_id, set_value)
        catalog.update_one(query, set_value)

In [28]:
def save_to_history(collection, doc_id, query):
    doc_history = f'{collection}_ID{doc_id}_History'
    count = r.zcard(doc_history)
    
    doc_dict = {}
    doc_dict[str(query)] = count + 1
    
    r.zadd(doc_history, doc_dict)

In [29]:
def undo(collection, doc_id):
    doc_history = f'{collection}_ID{doc_id}_History'
    doc = collection.find_one(doc_id)
    
    try:
        count = r.zcard(doc_history) + 1
        query = {'id': doc_id}
        value = r.zrange(doc_history, count, count + 1)
        history = get_history(catalog, doc_id)
        print(history)
#         value = history([len(history) - 1])
        print(value)
#         value = json.loads(value)
        catalog.update_one(query, value)
        
        r.zremrangebyscore(doc_history, count, count + 1)
    except Exception as e:
        print(e)
#         print(f'{doc} does not have an edit history')

In [30]:
def get_history(collection, doc_id):
    doc_history = f'{collection}_ID{doc_id}_History'
    return r.zrange(doc_history, 0, -1)

In [31]:
undo(catalog, 2)

[b"{'$set': {'title': 'The Ugliest Boy in Town'}}", b"{'$set': {'title': 'Total Recall'}}", b"{'$set': {'title': 'Weeble Wobble'}}"]
[]
update cannot be empty


In [32]:
delete_book(2)

Book ID: 2 does not exist in our system


### Search by quote

In [33]:
def search_quote(quote):
    if not 'text_text' in texts.index_information():
        texts.create_index([('text', 'text')])
    try:
        book_content = texts.find({'$text': {'$search': f'\"{quote}\"'}}).limit(1)
        for item in book_content:
            book_id = item['_id']
        query = {'_id': book_id}
        book = catalog.find_one(query)
        return book
    except UnboundLocalError:
        print('Cound not find your quote')

In [34]:
quote_1 = "Brevity is the soul of wit"
quote_2 = "God is dead"

search_quote(quote_1)

{'_id': 33186,
 'title': 'Hamlet  Prince of Denmark',
 'author': 'William Shakespeare/Philip Edwards/Robert Hapgood',
 'average_rating': 4.02,
 'isbn': '521532523',
 'language_code': 'eng',
 'num_pages': 270,
 'ratings_count': 149,
 'text_reviews_count': 13,
 'publication_date': '4/21/2003',
 'publisher': 'Cambridge University Press/The New Cambridge Shakespeare',
 'available': 1}

In [35]:
search_quote(quote_2)

{'_id': 18310,
 'title': 'Thus Spake Zarathustra: A Book for All and None',
 'author': 'Friedrich Nietzsche',
 'average_rating': 4.06,
 'isbn': '1406947199',
 'language_code': 'eng',
 'num_pages': 356,
 'ratings_count': 275,
 'text_reviews_count': 20,
 'publication_date': '11/3/2006',
 'publisher': 'Penguin',
 'available': 1}

## Employee SCUM

In [36]:
def get_employee(employee_id):
    query = {'id': employee_id}
    employee = employees.find_one(query)
    return employee

In [37]:
def add_employee(employee_id, surname, first_name, schedule,weekly_hours,
                 wage_type, pay, phone, email, password):
        try:
            this_employee = {'_id': employee_id,
                            'surname': surname,
                            'firstName': first_name,
                            'schedule': schedule,
                            'weeklyHours': weekly_hours,
                            'wageType': wage_type,
                            'pay': pay,
                            'phone': phone,
                            'email': email,
                            'username': username,
                            'password': password
                            }
            employees.insert_one(this_book)
            print(f'Sucessfully added {first_name} {surname} to system.')
        except:
            print(f'Employee ID already exists in our system.')

In [38]:
def delete_employee(employee_id):
    try:
        query = {'_id': employee_id}
        employees.delete_one(query)
    except:
        print(f'Employee ID: {employee_id} does not exist in our system.')

In [39]:
def update_employee_field(employee_id, field, value):
        query = {'_id': employee_id}
        set_value = {'$set': {field : value}}
        catalog.update_one(query, set_value)

## Patron functions

### Basic SCUM

In [40]:
def get_patron(patron_id):
    query = {'_id': patron_id}
    patron = patrons.find_one(query)
    return patron

In [41]:
def add_patron(patron_id, surname, first_name, address, phone, email, username, password, loc, pub_priv):
    try:
        this_patron = {'_id': patron_id,
                      'surname': surname,
                      'firstName': first_name,
                      'address': address,
                      'phone': phone,
                      'email': email,
                      'username': username,
                      'password': password,
                      'loc': loc,
                      'pubPriv': pub_priv}
        patrons.insert_one(this_patron)
    except:
        print('Patron ID already exists in this system')

In [42]:
def delete_patron(patron_id):
    try:
        query = {'_id': patron_id}
        patrons.delete_one(query)
    except:
        print(f'Patron ID: {patron_id} does not exist in our system')

In [43]:
def delete_patrons(patronID_list):
    for patron_id in patronID_list:
        delete_patron(patron_id)

In [44]:
def update_patron_field(patron_id, field, value):
    query = {'_id': patron_id}
    set_value = {'$set': {field : value}}
    patrons.update_one(query, set_value)

In [45]:
def favorite(patron_id, book_id):
    query = {'_id': patron_id}
    book = get_book(book_id)
    patron_book = {'$push': {'favorites': book}}
    patrons.update_one(query, patron_book)


In [46]:
def unfavorite(patron_id, book_id):
    query = {'_id': patron_id}
    book = get_book(book_id)
    
    patron_book = {'$pull': {'favorites': book}}
    patrons.update_one(query, patron_book)

#### Demo

In [47]:
favorite(1, 4)

In [48]:
unfavorite(1, 4)

In [49]:
get_patron(1)

{'_id': 1,
 'surname': 'Estrada',
 'firstName': 'Heather',
 'address': '1912 Howard Bypass Apt. 110, Roscoe, IL, 61080',
 'phone': '(580)417-1971',
 'email': 'heestrada@gmail.com',
 'username': 'heestrada',
 'password': 'BbHt4h3Afg',
 'loc': [-89.03274288039283, 42.449495529630994],
 'pubPriv': 'private',
 'checkedOutBooks': [],
 'favorites': []}

### Check-out and Check-in

In [50]:
def add_to_cart(patron_id, book_id):
    book = catalog.find_one({'_id': book_id})
    
    cart = f'Patron_{patron_id:05}_Cart'
    count = r.hlen(cart)
    try:
        if book['available'] == 0:
           print(f'Sorry, "{book["title"]}" is not currently available')
        elif bytes(str(book_id), 'utf-8') not in r.hscan(cart)[1].values():
            r.hset(cart, f'book_{count + 1:0}', book_id)
            print(f'added "{book["title"]}" to your cart')
        else:
            print(f'you already have "{book["title"]}" in your cart')
    except TypeError:
        print(f'Sorry, a book with bookID: {book_id} is not in our system')

In [51]:
def check_out(patron_id):
    cart = r.hscan(f'Patron_{patron_id:05}_Cart')[1]
    
    for book_id in cart.values():
        book_id = int(book_id)
        
        cat_query = {'_id': book_id}
        checkout = {'$set': {'available': 0}}
        catalog.update_one(cat_query, checkout)
        
        pat_query = {'_id': patron_id}
        patron_book = {'$push': {'checkedOutBooks': book_id}}
        
        patrons.update_one(pat_query, patron_book)
    
    r.delete(f'Patron_{patron_id:05}_Cart')
    

In [52]:
def check_in(patron_id, book_id_list):
    for book_id in book_id_list:
        cat_query = {'_id': book_id}
        pat_query = {'_id': patron_id}

        patron = patrons.find_one(pat_query)
        booklist= patron['checkedOutBooks']

        book = catalog.find_one(cat_query)

        if book_id in booklist:

            checkin = {'$set': {'available': 1}}
            catalog.update_one(cat_query, checkin)

            book_query = {'$pull': {'checkedOutBooks': book_id}}
            patrons.update_one(pat_query, book_query)
        else:
            print(f'Patron {patron["firstName"]} {patron["surname"]} does not have "{book["title"]}" checked out.')

In [53]:
def get_patron_checked_out(patron_id):
    patron = get_patron(patron_id)
    checked_out = patron['checkedOutBooks']
    return checked_out

#### Demo

In [54]:
add_to_cart(1, 2)
add_to_cart(1, 3)
add_to_cart(1, 130)
add_to_cart(1, 45)
add_to_cart(1, 30)

Sorry, a book with bookID: 2 is not in our system
Sorry, a book with bookID: 3 is not in our system
added "Power of an Hour: Business and Life Mastery in One Hour a Week" to your cart
added "Agile Web Development with Rails: A Pragmatic Guide" to your cart
added "J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings" to your cart


In [55]:
check_out(1)
get_patron_checked_out(1)

[130, 45, 30]

In [56]:
book_list = get_patron_checked_out(1)
check_in(1, book_list)

### Patrons Journals

In [57]:
texts.create_index([('text', pym.TEXT)], default_language='english')

'text_text'

In [58]:
def save_book_entry(patron_id, book_id, entry):
    book = get_book(book_id)   
    journal = f'Patron_{patron_id:05}_Journal'
    count = r.zcard(journal)
    entry_dict = {}
    entry_dict[f'{book["title"]}: {entry}'] = count + 1
    
    r.zadd(journal, entry_dict, nx=True)

In [59]:
def write_note(patron_id, entry):
    journal = f'Patron_{patron_id:05}_Journal'
    count = r.zcard(journal)
    entry_dict = {}
    entry_dict[f'Personal Note: {entry}'] = count + 1
    
    r.zadd(journal, entry_dict, nx=True)

In [60]:
def get_journal(patron_id):
    journal = f'Patron_{patron_id:05}_Journal'
    content = r.zrange(journal, 0, -1)
    return content

In [61]:
def delete_journal_entry(patron_id, entry):
    journal = f'Patron_{patron_id:05}_Journal'
    r.zrem(journal,f'Entry_{entry}')

In [62]:
def delete_journal(patron_id):
    journal = f'Patron_{patron_id:05}_Journal'
    r.delete(journal)

#### Demo

In [63]:
save_book_entry(1, 30474, "The bourgeoisie cannot exist without constantly revolutionising the instruments of production, and thereby the relations of production, and with them the whole relations of society.")

In [64]:
write_note(1, "Someday I'LL write a book")
write_note(1, "Or I'll write a poem")
write_note(1, "Someday, I'll write a sentence")

In [65]:
save_book_entry(1, 10534, "The supreme art of war is to subdue the enemy without fighting.")

In [66]:
journal = get_journal(1)
journal

[b'The Communist Manifesto: The bourgeoisie cannot exist without constantly revolutionising the instruments of production, and thereby the relations of production, and with them the whole relations of society.',
 b"Personal Note: Someday I'LL write a book",
 b"Personal Note: Or I'll write a poem",
 b"Personal Note: Someday, I'll write a sentence",
 b'The Art of War: The supreme art of war is to subdue the enemy without fighting.']

In [67]:
delete_journal(1)

### Patron Location and Sharing

In [68]:
def get_location(patron_id):
    pat_query = {'_id': patron_id}
    patron = patrons.find_one(pat_query)
    print(patron['loc'])

In [69]:
def update_location(patron_id, long, lat):
    pat_query = {'_id': patron_id}
    patron = patrons.find_one(pat_query)
    
    update_loc = {'$set': {'loc': [long, lat]}}
    
    patrons.update_one(pat_query, update_loc)

In [70]:
def toggle_loc_share(patron_id):
    pat_query = {'_id': patron_id}
    patron = patrons.find_one(pat_query)
    if patron['pubPriv'] == 'public':
        set_priv = {'$set': {'pubPriv': 'private'}}
        patrons.update_one(pat_query, set_priv)
        print(f'Patron {patron["_id"]} set to "private"')
    else:
        set_pub = {'$set': {'pubPriv': 'public'}}
        patrons.update_one(pat_query, set_pub)
        print(f'Patron {patron["_id"]} set to "public"')

In [71]:
def find_nearby_patrons(patron_id):
    patron = get_patron(patron_id)
    cardinal = ['North', 'East', 'South', 'West']
    nearby_patrons = []
    report = []

    for near_patron in patrons.find({"loc": {"$within": {"$center": [patron['loc'], .01]}}}).limit(20):
        if near_patron['pubPriv'] == 'public' and near_patron['_id'] != patron['_id']:
            lat = ((patron['loc'][0] - near_patron['loc'][0]) * 139)
            long = ((patron['loc'][1] - near_patron['loc'][1]) * 111)
            
            nearby_patrons.append(near_patron['_id'])
            report.append(f'Patron {near_patron["firstName"]} {near_patron["surname"]} (ID:{near_patron["_id"]}) is {abs(long) * 10} meters {cardinal[0] if long > 0 else cardinal[2]} and {abs(lat)*10} meters {cardinal[1] if lat > 0 else cardinal[3]}')
    return nearby_patrons, report

#### Demo

In [72]:
nearby, report = find_nearby_patrons(1)
for patron in report:
    print(patron)

Patron Martin Meyers (ID:428) is 1.9037465201707704 meters North and 9.316445081107787 meters East
Patron Taylor Bryant (ID:110) is 0.09539039337752797 meters South and 9.773075715985868 meters East
Patron Ashley Wells (ID:278) is 9.602758194364753 meters North and 5.086691589408048 meters East
Patron Jeffrey Reed (ID:520) is 9.616877470664775 meters North and 4.4989690780043645 meters East
Patron Tracy Dominguez (ID:872) is 8.34546151464167 meters North and 4.589582136346166 meters East
Patron Jesse Hale (ID:241) is 4.7732143657975 meters North and 1.9125025445720212 meters East
Patron Charles Jordan (ID:343) is 0.9563625038899204 meters North and 5.669658470117014 meters East
Patron Robert Nelson (ID:126) is 0.2991466549914179 meters South and 2.048750956911931 meters East
Patron Dylan Miller (ID:358) is 3.606111335604112 meters South and 12.857871977741553 meters East
Patron Jennifer Miller (ID:634) is 0.5705487576206281 meters South and 0.10102804150477596 meters East
Patron Collee

In [73]:
toggle_loc_share(246)

Patron 246 set to "public"


In [74]:
get_location(2)

[-88.97699961874508, 42.40738449796045]


In [75]:
update_location(2,-89.061, 42.4627)

### Book Club

In [76]:
def send_message(messenger, recipient, message):
    if messenger['pubPriv'] == 'private' :
        error_message = 'You need to set your account to public before you can message others'
    elif recipient['pubPriv'] == 'private':
        error_message = 'Sorry, that patron has their account set to private'

    elif not r.exists(f'{recipient["surname"]}_{messenger["surname"]}_Conversation'):
        conversation = f'{messenger["surname"]}_{recipient["surname"]}_Conversation'
    else:
        conversation = f'{recipient["surname"]}_{messenger["surname"]}_Conversation'
    
    try:
        count = r.zcard(conversation) + 1
        message_dict = {}
        message_dict[f'{count}. {messenger["firstName"]} says: {message}'] = count

        r.zadd(conversation, message_dict)
    except:
        print(error_message)

In [77]:
def get_conversation(patron_1, patron_2):
    if not r.exists(f'{patron_1["surname"]}_{patron_2["surname"]}_Conversation'):
        conversation = r.zrange(f'{patron_2["surname"]}_{patron_1["surname"]}_Conversation', 0, -1)
    else:
        conversation = r.zrange(f'{patron_1["surname"]}_{patron_2["surname"]}_Conversation', 0, -1)
    return conversation

In [78]:
def delete_conversation(patron_1, patron_2):
    if not r.exists(f'{patron_1["surname"]}_{patron_2["surname"]}_Conversation'):
        r.delete(f'{patron_2["surname"]}_{patron_1["surname"]}_Conversation')
    else:
        r.delete(f'{patron_1["surname"]}_{patron_2["surname"]}_Conversation')

In [79]:
def create_popUp_bookClub(creator, club_name):
    club_name = f'The {club_name} Club'
    if r.exists(club_name):
        print(f'{club_name} already exists.')
    else:
        nearby = find_nearby_patrons(creator['_id'])[0]
        count = r.zcard(club_name)

        club_dict = {}
        club_dict[f'{count + 1}. Patron {creator["firstName"]} {creator["surname"]} has created {club_name}!'] = count + 1

        r.zadd(club_name, club_dict)
        for recipient_id in nearby:
            recipient = get_patron(recipient_id)
            send_message(creator, recipient, f'Just created {club_name}! Setting up at {creator["loc"]}')

In [80]:
def message_club(messenger, club_name, message):
    club_name = f'The {club_name} Club'
    count = r.zcard(club_name)
    message_dict = {}
    message_dict[f'{count + 1}. {messenger["firstName"]} says: {message}'] = count + 1
    if count != 0:
        r.zadd(club_name, message_dict)
    else:
        print(f'{club_name} does not exist.')

In [81]:
def get_club_log(club_name):
    club_name = f'The {club_name} Club'
    return r.zrange(club_name, 0, -1)
    

In [82]:
def delete_club(club_name):
    club = f'The {club_name} Club'
    r.delete(club)

#### Demo

In [83]:
toggle_loc_share(5) # Use this to set patron_1 to "public"

Patron 5 set to "private"


In [84]:
club = 'Sci-fi'
patron_1 = get_patron(5)
patron_2 = get_patron(645)
patron_3 = get_patron(234)

create_popUp_bookClub(patron_1, club)
message_club(patron_2, club, 'interested in meeting up!')
message_club(patron_3, club, 'Me three!')

get_club_log(club)

You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others
You need to set your account to public before you can message others


[b'1. Patron Michele Andrade has created The Sci-fi Club!',
 b'2. Jennifer says: interested in meeting up!',
 b'3. Ryan says: Me three!']

In [85]:
delete_club(club)
get_club_log(club)

[]

In [86]:
nearby, report = find_nearby_patrons(5)
for patron in report:
    print(patron)

Patron Colleen Peterson (ID:418) is 4.666894861204369 meters North and 10.544457906269571 meters East
Patron Lisa Mendoza (ID:185) is 5.734455529613811 meters North and 6.7285465377001685 meters East
Patron Jennifer Potts (ID:645) is 6.934723904587656 meters South and 9.21943188562409 meters East
Patron Craig Murray (ID:517) is 9.888678841228113 meters North and 3.475235001300234 meters East
Patron Steven Clark (ID:149) is 5.955999928402562 meters North and 1.4564028356045355 meters West
Patron Andrew Mays (ID:528) is 7.9320548208342245 meters North and 3.6934904667859314 meters West
Patron Michelle Erickson (ID:411) is 8.55645883176372 meters North and 5.016531514269644 meters West
Patron Jeff Williams (ID:502) is 5.755069248390541 meters North and 8.949702256709031 meters West
Patron Alan Clark (ID:172) is 3.4444236044625143 meters North and 1.3503219017310641 meters West
Patron Robin Gill (ID:335) is 2.0215752056895298 meters South and 0.529112415235744 meters East
Patron Jeremy Mil

In [87]:
toggle_loc_share(5)

Patron 5 set to "public"


In [88]:
patron_1 = get_patron(5)
patron_2 = get_patron(645)

send_message(patron_2, patron_1, 'Have you read "Blindsight"? I have been trying to find someone who has')
send_message(patron_1, patron_2, 'I have!')
send_message(patron_2, patron_1, 'Great! See you soon!')

get_conversation(patron_1, patron_2)

[b'1. Jennifer says: Have you read "Blindsight"? I have been trying to find someone who has',
 b'2. Michele says: I have!',
 b'3. Jennifer says: Great! See you soon!']

In [89]:
delete_conversation(patron_1, patron_2)
get_conversation(patron_1, patron_2)

[]

In [90]:
r.flushall()

True